In [1]:
'''
Import based on imports found in the notebooks located here:
Schivley Greg, PowerGenome, (2022), GitHub repository, 
    https://github.com/PowerGenome/PowerGenome/tree/master/notebooks
'''


import pandas as pd
import numpy as np

###
# module issue
import os
import sys
module_path = os.path.abspath(os.getcwd() + '\\..')
if module_path not in sys.path:
    sys.path.append(module_path)
###
from pathlib import Path
from powergenome.util import (
    build_scenario_settings,
    init_pudl_connection,
    load_settings,
    check_settings
)

cwd = Path.cwd()

settings_path = (cwd / "settings_TD.yml")
settings = load_settings(settings_path)

In [2]:
IPM_regions = settings.get('model_regions')
aeo_fuel_region_map = settings.get('aeo_fuel_region_map')
# aeo_fuel_region_map

In [3]:
# read in csv outputs from PG
#existing_gen = pd.read_csv('PG_output_csv/existing_gen_WECC.csv', index_col=0)
#new_gen = pd.read_csv('PG_output_csv/new_gen_WECC.csv', index_col=0)
#existing_variability = pd.read_csv('PG_output_csv/existing_variability_WECC.csv', index_col=0)
#potential_build_yr = pd.read_csv('PG_output_csv/gc_units_model_WECC.csv', index_col=0)
# all_gen = pd.read_csv('PG_output_csv/all_gen_WECC.csv', index_col=0)
# fuels = pd.read_csv('PG_output_csv/fuels_WECC.csv', index_col=0)
fuel_prices = pd.read_csv('PG_output_csv/fuel_prices_WECC.csv', index_col=0)
#load_curves = pd.read_csv('PG_output_csv/load_curves_WECC.csv', index_col=0)

In [4]:
# fuel_prices

In [5]:
def fuel_cost_table(aeo_fuel_region_map, fuel_prices, IPM_regions, scenario, year_list):
    '''
    Create the fuel_cost input file based on REAM Scenario 178.
    Inputs:
        * aeo_fuel_region_map: has aeo_fuel_regions and the ipm regions within each aeo_fuel_region
        * fuel_prices: output from PowerGenome gc.fuel_prices
        * IPM_regions: from settings('model_regions')
        * scenario: filtering the fuel_prices table. Suggest using 'reference' for now.
        * year_list: the periods - 2020, 2030, 2040, 2050.  To filter the fuel_prices year column
    Output:
        the fuel_cost_table
            * load_zone: IPM region
            * fuel: based on PowerGenome fuel_prices table
            * period: based on year_list
            * fuel_cost: based on fuel_prices.price
    '''
    
    ref_df = fuel_prices.copy()
    ref_df = ref_df.loc[fuel_prices['scenario']==scenario] # use reference scenario for now
    ref_df = ref_df[ref_df['year'].isin(year_list)]
    ref_df = ref_df.drop(['full_fuel_name','scenario'], axis=1)
    
    # loop through aeo_fuel_regions. 
    # for each of the ipm regions in the aeo_fuel, duplicate the fuel_prices table while adding ipm column
    fuel_cost = pd.DataFrame(columns=['year','price','fuel','region', 'load_zone'])
    data = list()
    for region in aeo_fuel_region_map.keys():
        df = ref_df.copy()
        df = df[df['region']==region]
        for ipm in aeo_fuel_region_map[region]:
            ipm_region = ipm
            df['load_zone'] = ipm_region
            fuel_cost = fuel_cost.append(df)
#     fuel_cost = fuel_cost.append(data)
    fuel_cost.rename(columns = {'year':'period', 'price':'fuel_cost'}, inplace = True)
    fuel_cost = fuel_cost[['load_zone', 'fuel', 'period', 'fuel_cost']]
    fuel_cost['period'] = fuel_cost['period'].astype(int)
    fuel_cost = fuel_cost[fuel_cost['load_zone'].isin(IPM_regions)]
    fuel_cost['fuel'] = fuel_cost['fuel'].str.capitalize() # align with energy_source in gen_pro_info? switch error.
    return fuel_cost

In [6]:
fuel_cost = fuel_cost_table(aeo_fuel_region_map, fuel_prices, IPM_regions, scenario='reference', year_list=
                            [2020,2030,2040,2050])

fuel_cost

,load_zone,fuel,period,fuel_cost
0,WECC_AZ,Coal,2050,1.439946
10,WECC_AZ,Coal,2040,1.426481
20,WECC_AZ,Coal,2030,1.402760
30,WECC_AZ,Coal,2020,1.745610
160,WECC_AZ,Naturalgas,2050,4.624599
...,...,...,...,...
990,WEC_SDGE,Distillate,2020,21.177547
1120,WEC_SDGE,Uranium,2050,0.700255
1130,WEC_SDGE,Uranium,2040,0.681301
1140,WEC_SDGE,Uranium,2030,0.664344


In [7]:
fuel_cost.to_csv(r'SWITCH_Inputs\fuel_cost.csv', index = False)